対象設定

In [13]:
name="mimikkyu"

デフォルトマーカー位置設定

In [14]:
import numpy as np 
import cv2
import glob
import os

aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
dst=cv2.imread( "./img/"+"/md_"+str(name)+".png") 
def_corners, def_ids, rejectedImgPoints = aruco.detectMarkers(dst, dictionary)

count=[0,1,2,3]
count2=[0,1,2,3,4,5,6,7]

def_pts=np.zeros((8,4,2),dtype = np.float32)

for i in count2:
    for j in count2:
        if j ==def_ids[i][0]:
            for k in count:
                def_pts[j][k][0]=def_corners[i][0][k][0]
                def_pts[j][k][1]=def_corners[i][0][k][1]
 
        
height, width, color = dst.shape


def_bord_pts=np.zeros((16,2),dtype = np.float32)
def_pro_pts=np.zeros((16,2),dtype = np.float32)

count=[0,1,2,3]
def_pts
for i in count:
    for j in count:
        def_bord_pts[4*i+j][0]=def_pts[i][j][0]
        def_bord_pts[4*i+j][1]=def_pts[i][j][1]    

for i in count:
    for j in count:
        def_pro_pts[4*i+j][0]=def_pts[4+i][j][0]
        def_pro_pts[4*i+j][1]=def_pts[4+i][j][1]



貼り付け関数

In [15]:
def cvpaste(img, imgback, x, y, angle, scale):  
    # x and y are the distance from the center of the background image 

    r = img.shape[0]
    c = img.shape[1]
    rb = imgback.shape[0]
    cb = imgback.shape[1]
    hrb=round(rb/2)
    hcb=round(cb/2)
    hr=round(r/2)
    hc=round(c/2)

    # Copy the forward image and move to the center of the background image
    imgrot = np.zeros((rb,cb,3),np.uint8)
    imgrot[hrb-hr:hrb+hr,hcb-hc:hcb+hc,:] = img[:hr*2,:hc*2,:]

    # Rotation and scaling
    M = cv2.getRotationMatrix2D((hcb,hrb),angle,scale)
    imgrot = cv2.warpAffine(imgrot,M,(cb,rb))
    # Translation
    M = np.float32([[1,0,x],[0,1,y]])
    imgrot = cv2.warpAffine(imgrot,M,(cb,rb))

    # Making mask
    imggray = cv2.cvtColor(imgrot,cv2.COLOR_BGR2GRAY)
    ret, mask = cv2.threshold(imggray, 10, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Now black-out the area of the forward image in the background image
    img1_bg = cv2.bitwise_and(imgback,imgback,mask = mask_inv)

    # Take only region of the forward image.
    img2_fg = cv2.bitwise_and(imgrot,imgrot,mask = mask)

    # Paste the forward image on the background image
  #  img1_bg2 = cv2.cvtColor(imgrot,cv2.COLOR_GRAY2GBGR)
    imgpaste = cv2.add(img1_bg,img2_fg)

    return imgpaste


メイン関数

In [16]:
import cv2
import numpy as np
import time
from pathlib import Path

aruco = cv2.aruco
dictionary = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
img=cv2.imread("img/"+str(name)+".jpg")
imgback = cv2.imread("img/"+str(name)+".jpg")

img_paths = Path("img/deformation/scalling_diff_BGR") / "%04d.png"

height,width,color=imgback.shape

angle=0
scale=0.2

ar4=cv2.imread("ARUCO/BGR2GRAY_ar04.png")
ar5=cv2.imread("ARUCO/BGR2GRAY_ar05.png")
ar6=cv2.imread("ARUCO/BGR2GRAY_ar06.png")
ar7=cv2.imread("ARUCO/BGR2GRAY_ar07.png")

cap = cv2.VideoCapture(1)
cap2 = cv2.VideoCapture(str(img_paths))

rt_pts=np.zeros((8,4,2),dtype = np.float32)
rt_bord_pts=np.zeros((16,2),dtype = np.float32)
rt_pro_pts=np.zeros((16,2),dtype = np.float32)

count=[0,1,2,3]
count2=[0,1,2,3,4,5,6,7]
count3=[4,5,6,7]

mat1, mask1 = cv2.findHomography(def_pro_pts, def_pro_pts, cv2.RANSAC,1)#初期設定（変更なし）
mat2, mask2 = cv2.findHomography(def_bord_pts, def_bord_pts, cv2.RANSAC,1)
frag=1#待機時間管理

while(True):

    t0 = time.time() #時間計測開始   
    # Capture frame-by-frame
    ret, frame = cap.read()
    ret_img, frame_img = cap2.read()
    
    if ret_img==True:

    # Display the resulting frame
        cv2.imshow('frame', frame)
        frame2=cv2.warpPerspective(frame_img, mat2, (3*width, 3*height))
        frame3=cv2.warpPerspective(frame2, mat1, (width, height))#カメラから見たプロジェクタ上の座標→カメラから見たボード上の座標
    

        imgpaste = cvpaste(ar4, frame3, 0, -90, angle, scale)#AR貼り付け
        imgpaste2 = cvpaste(ar5, imgpaste, 180, 0, angle, scale)
        imgpaste3 = cvpaste(ar6, imgpaste2, 0, 90, angle, scale)
        imgpaste4 = cvpaste(ar7, imgpaste3, -180, 0, angle, scale)

    

        cv2.namedWindow('screen', cv2.WINDOW_NORMAL)
        cv2.setWindowProperty('screen', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
        cv2.imshow('screen',imgpaste4)
  
    else:
        cap2.set(cv2.CAP_PROP_POS_FRAMES, 0)

    new_corners, new_ids, rejectedImgPoints = aruco.detectMarkers(frame, dictionary)
    
    if not new_corners:
        pass
    
#    elif new_ids.size<=4:
 #       pass
        
    else:
        #rt_pts=np.zeros((8,4,2),dtype = np.float32)
        for i in count:
            for j in count:
                    rt_pts[i+4][j][0]=0
                    rt_pts[i+4][j][1]=0
            
        rt_bord_pts=np.zeros((16,2),dtype = np.float32)#reset
        rt_pro_pts=np.zeros((16,2),dtype = np.float32)
        def_bord_pts2=np.zeros((16,2),dtype = np.float32)
        def_pro_pts2=np.zeros((16,2),dtype = np.float32)
        num=new_ids.size-1
        
        for i in count2:
            for j in count2:
                if j ==new_ids[i][0]:
                    for h in count:
                        rt_pts[j][h][0]=new_corners[i][0][h][0]                           
                        rt_pts[j][h][1]=new_corners[i][0][h][1]
            if num==i:
                   break
            

        k=0
        for i in count:
            if rt_pts[i][0][0]==rt_pts[i][0][1]==0:
                for j in count:
                    def_bord_pts2=np.delete(def_bord_pts2,-1,0)
                    rt_bord_pts=np.delete(rt_bord_pts,-1,0)

                    
            else:
                for j in count:
                    def_bord_pts2[4*k+j][0]=def_bord_pts[4*i+j][0]
                    def_bord_pts2[4*k+j][1]=def_bord_pts[4*i+j][1] 
                    rt_bord_pts[4*k+j][0]=rt_pts[i][j][0]
                    rt_bord_pts[4*k+j][1]=rt_pts[i][j][1] 

                k=k+1
                        
        k=0
       # print(rt_pts)
        for i in count:
            if rt_pts[i+4][0][0]==rt_pts[i+4][0][1]==0:
                for j in count:
                    def_pro_pts2=np.delete(def_pro_pts2,-1,0)
                    rt_pro_pts=np.delete(rt_pro_pts,-1,0)
                  #  print("deleted")
                    
            else:
                for j in count:
                    def_pro_pts2[4*k+j][0]=def_pro_pts[4*i+j][0]
                    def_pro_pts2[4*k+j][1]=def_pro_pts[4*i+j][1]
                    rt_pro_pts[4*k+j][0]=rt_pts[i+4][j][0]
                    rt_pro_pts[4*k+j][1]=rt_pts[i+4][j][1]
                k=k+1
        if  rt_pro_pts.size==0 or rt_bord_pts.size==0 : 
            pass
            
        elif frag>=30:#待機時間管理
            mat1, mask1 = cv2.findHomography(rt_pro_pts, def_pro_pts2, cv2.RANSAC,1)
            mat2, mask2 = cv2.findHomography(def_bord_pts2, rt_bord_pts, cv2.RANSAC,1)
    #print(cap.get(cv2.CAP_PROP_FPS))


    if cv2.waitKey(1) &0xFF == ord('q'):
        break
    frag=frag+1
    t1 = time.time()                # 計測終了時間
    elapsed_time = float(t1 - t0)   # 経過時間
   # print(elapsed_time)             # 経過時間を表示
    
cv2.destroyAllWindows()
cap2.release()
cap.release()

差分画像生成

In [9]:
import cv2 
import numpy as np 
import os
from datetime import datetime

img = cv2.imread("img/"+str(name)+".jpg") 
h,w,color=img.shape

n=0
i=0
t=1/w

while True:
    pts1 = np.float32([[0,0],[w,0],[0,h]])
    pts2 = np.float32([[0-w*t,0-h*t],[w+w*t,0-h*t],[0-w*t,h+h*t]])

    mat = cv2.getAffineTransform(pts1,pts2)
    transed_img = cv2.warpAffine(img, mat, (w, h))
    cv2.imwrite("./img/deformation/scalling/" + str(i) + ".png", transed_img)

    n+=1
    t+=1/w
    i+=1


    if n==2:#秒数によって変える
        break    

while True:
    pts1 = np.float32([[0,0],[w,0],[0,h]])
    pts2 = np.float32([[0-w*t,0-h*t],[w+w*t,0-h*t],[0-w*t,h+h*t]])

    mat = cv2.getAffineTransform(pts1,pts2)
    transed_img = cv2.warpAffine(img, mat, (w, h))
    cv2.imwrite("./img/deformation/scalling/" + str(i) + ".png", transed_img)

    n+=1
    t=t-1/w
    i+=1

    if n==5:#秒数によって変える
        break
    
t=t-1/h
        
while True:
    pts1 = np.float32([[0,0],[w,0],[0,h]])
    pts2 = np.float32([[0-w*t,0-h*t],[w+w*t,0-h*t],[0-w*t,h+h*t]])

    mat = cv2.getAffineTransform(pts1,pts2)
    transed_img = cv2.warpAffine(img, mat, (w, h))

    cv2.imwrite("./img/deformation/scalling/" + str(i) + ".png", transed_img)
    n+=1
    t=t-1/w
    i+=1
    if n==7:#秒数によって変える
        break
    
while True: 
    pts1 = np.float32([[0,0],[w,0],[0,h]])
    pts2 = np.float32([[0-w*t,0-h*t],[w+w*t,0-h*t],[0-w*t,h+h*t]])

    mat = cv2.getAffineTransform(pts1,pts2)
    transed_img = cv2.warpAffine(img, mat, (w, h))
    cv2.imwrite("./img/deformation/scalling/" + str(i) + ".png", transed_img)
    n+=1
    t+=1/w
    i+=1
    if n==10:#秒数によって変える
        break


In [67]:
import cv2
import numpy as np

n=0
i=0

or_img=cv2.imread("img/"+str(name)+".jpg")
or_img_gray=cv2.cvtColor(or_img,cv2.COLOR_BGR2GRAY)

h,w,color=or_img.shape



while(True):
    i='%04d'% n  
    img=cv2.imread("./img/deformation/scalling/" + str(n) + ".png")
    img_gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    im_diff_gray=or_img_gray.astype(int)-img_gray.astype(int)
    im_diff_center=np.floor_divide(im_diff_gray, 2) + 128
    im_diff_abs=np.abs(im_diff_center)
    img_white = np.ones((h,w),np.uint8)
    for k in range(h):
        for j in range(w):
            img_white[k][j]=float(im_diff_abs[k][j])
            
    im_diff=cv2.cvtColor(img_white,cv2.COLOR_GRAY2BGR)

    path = "./img/deformation/scalling_diff_BGR/" + str(i) + ".png"
    
    cv2.imwrite(path, im_diff) # ファイル保存
    n+=1
    if n==10:#秒数によって変える
        break


In [66]:
float(im_diff_abs[24][23])

128.0

In [21]:
or_img.shape

(225, 400, 3)

In [55]:
h

225

In [59]:
w

400